In [84]:
import os
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox, Toplevel, simpledialog
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
from tkinter import ttk

# Charger le fichier CSV contenant les informations prédéfinies
csv_path = 'Master2_04_01_2025.csv'  # Remplacez par le chemin réel de votre fichier CSV

# Charger les données et vérifier les colonnes nécessaires
try:
    data = pd.read_csv(csv_path)
    required_columns = {'image_file', 'label', 'Age', 'classification'}
    if not required_columns.issubset(data.columns):
        raise ValueError(f"Le fichier CSV doit contenir les colonnes suivantes : {required_columns}")
except FileNotFoundError:
    messagebox.showerror("Erreur", f"Le fichier {csv_path} n'a pas été trouvé. Vérifiez le chemin.")
    exit()
except Exception as e:
    messagebox.showerror("Erreur", f"Erreur lors du chargement des données : {str(e)}")
    exit()

# Mapping des classifications
classification_map = {0: 'Normal', 1: 'Benin', 2: 'Malignant'}

# Fonction pour récupérer les informations d'une image dans le CSV
def get_image_info(file_name):
    record = data[data['image_file'].str.contains(file_name, case=False, na=False)]
    if not record.empty:
        return record.iloc[0]
    return None

# Fonction pour afficher des informations sur l'application
def show_about():
    messagebox.showinfo(
        "À propos",
        "Application pour la détection de cancer du sein.\n\n"
        "Utilisez cette application pour afficher des statistiques générales et un diagramme circulaire basé sur les données."
    )

# Fonction pour afficher un résumé statistique
def show_stats():
    if data.empty:
        messagebox.showinfo("Info", "Aucune donnée disponible pour afficher des statistiques.")
        return

    stats_summary = data['label'].value_counts()
    total = data.shape[0]
    stats_text = f"Résumé des statistiques :\n\n"
    for label, count in stats_summary.items():
        percentage = (count / total) * 100
        stats_text += f"{label}: {count} ({percentage:.1f}%)\n"

    stats_text += f"\nTotal des enregistrements : {total}"

    stats_window = Toplevel(root)
    stats_window.title("Statistiques")
    stats_window.geometry("400x300")

    stats_label = tk.Label(stats_window, text=stats_text, font=("Arial", 12), justify="left")
    stats_label.pack(pady=20, padx=20)

# Fonction pour afficher un diagramme circulaire
def show_pie_chart():
    if data.empty:
        messagebox.showinfo("Info", "Aucune donnée disponible pour générer un diagramme.")
        return

    label_counts = data['label'].value_counts()
    percentages = (label_counts / label_counts.sum()) * 100

    # Génération du diagramme circulaire
    plt.figure(figsize=(8, 6))
    plt.pie(
        label_counts.values,
        labels=label_counts.index,
        autopct='%1.1f%%',
        startangle=90,
        wedgeprops={'edgecolor': 'black'}
    )
    plt.title('Répartition des labels en pourcentage')
    plt.tight_layout()
    plt.show()

# Fonction pour afficher un diagramme en bâtons de la distribution des âges
def show_bar_chart():
    if data.empty:
        messagebox.showinfo("Info", "Aucune donnée disponible pour générer un diagramme.")
        return

    # Vérifier si la colonne 'Age' contient des valeurs manquantes
    data_clean = data.dropna(subset=['Age'])

    # Générer l'histogramme de la distribution des âges
    plt.figure(figsize=(8, 6))
    plt.hist(data_clean['Age'], bins=10, edgecolor='black', color='skyblue')
    plt.title('Distribution des âges')
    plt.xlabel('Âge')
    plt.ylabel('Fréquence')
    plt.tight_layout()
    plt.show()

# Fonction pour sélectionner une image
def select_image():
    file_path = filedialog.askopenfilename(
        title="Sélectionnez une image",
        filetypes=[("Image files", "*.png;*.jpg;*.jpeg")]
    )
    if not file_path:
        messagebox.showinfo("Info", "Aucune image sélectionnée.")
        return

    if not file_path.lower().endswith(('png', 'jpg', 'jpeg')):
        messagebox.showerror("Erreur", "Veuillez sélectionner un fichier image valide (.png, .jpg, .jpeg).")
        return

    try:
        img = Image.open(file_path)
        img = img.resize((300, 300))
        img_tk = ImageTk.PhotoImage(img)
        panel.configure(image=img_tk)
        panel.image = img_tk

        selected_image_path.set(file_path)
        info_text.set("Image chargée. Cliquez sur 'Prédire' pour voir le diagnostic.")
    except Exception as e:
        messagebox.showerror("Erreur", f"Impossible de charger l'image : {str(e)}")

# Fonction pour prédire le type de cancer
def predict_image():
    file_path = selected_image_path.get()
    if not file_path:
        messagebox.showinfo("Erreur", "Aucune image sélectionnée. Veuillez sélectionner une image.")
        return

    file_name = os.path.basename(file_path)
    record = get_image_info(file_name)
    if record is not None:
        classification_code = int(record['classification'])
        classification = classification_map.get(classification_code, "Inconnu")
        label = record['label'] if pd.notna(record['label']) else "Inconnu"
        age = record['Age'] if pd.notna(record['Age']) else "Non spécifié"

        info_text.set(
            f"Nom de l'image : {file_name}\n"
            f"Type : {classification}\n"
            f"Âge : {age}\n"
            f"Label : {label}"
        )
        messagebox.showinfo(
            "Prédiction",
            f"Résultat : {classification}\nÂge : {age}\nLabel : {label}"
        )
    else:
        info_text.set("Aucune correspondance trouvée dans la base de données.")
        messagebox.showerror("Erreur", "Aucune correspondance trouvée pour l'image sélectionnée.")

# Fonction de sauvegarde des résultats
def save_results():
    file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
    if file_path:
        try:
            with open(file_path, 'w') as file:
                file.write("Image,Classification,Age,Label\n")
                file.write(f"{os.path.basename(selected_image_path.get())},{info_text.get()}\n")
            messagebox.showinfo("Succès", "Résultats sauvegardés avec succès.")
        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur lors de la sauvegarde des résultats : {e}")

# Fonction pour réinitialiser les champs
def reset():
    panel.configure(image='')
    info_text.set('')
    selected_image_path.set('')
    messagebox.showinfo("Réinitialiser", "Les champs ont été réinitialisés.")

# Fonction de recherche d'image
def search_image():
    search_term = simpledialog.askstring("Recherche", "Entrez le nom de l'image :")
    if search_term:
        record = data[data['image_file'].str.contains(search_term, case=False, na=False)]
        if not record.empty:
            info_text.set(f"Image trouvée : {record.iloc[0]['image_file']}")
            # Affiche l'image correspondante ou d'autres informations
        else:
            messagebox.showinfo("Résultat", "Aucune image trouvée avec ce nom.")

# Fonction de connexion
def login():
    username = username_entry.get()
    password = password_entry.get()

    # Vérification des informations de connexion (ici, un exemple simple)
    if username == "admin" and password == "cancer":
        login_window.destroy()
        create_main_window()
    else:
        messagebox.showerror("Erreur", "Nom d'utilisateur ou mot de passe incorrect.")

# Fonction pour créer la fenêtre principale
def create_main_window():
    global root, panel, selected_image_path, info_text

    root = tk.Tk()
    root.title("Détection de type de cancer du sein")
    root.geometry("900x800")
    
    # Modifier la couleur de fond de la fenêtre principale
    root.configure(bg="#FFC0CB")  # Arrière-plan rose

    selected_image_path = tk.StringVar()
    info_text = tk.StringVar()

    # Barre de navigation avec fond rose
    nav_bar = tk.Frame(root, bg="#FFC0CB")
    nav_bar.pack(side="top", fill="x")

    btn_about = tk.Button(nav_bar, text="À propos", command=show_about, font=("Arial", 12), bg="#FFC0CB", fg="white", relief="flat")
    btn_about.pack(side="left", padx=10)

    btn_stats = tk.Button(nav_bar, text="Statistiques", command=show_stats, font=("Arial", 12), bg="#FFC0CB", fg="white", relief="flat")
    btn_stats.pack(side="left", padx=10)

    btn_pie_chart = tk.Button(nav_bar, text="Diagramme Circulaire", command=show_pie_chart, font=("Arial", 12), bg="#FFC0CB", fg="white", relief="flat")
    btn_pie_chart.pack(side="left", padx=10)

    btn_bar_chart = tk.Button(nav_bar, text="Diagramme en Bâtons", command=show_bar_chart, font=("Arial", 12), bg="#FFC0CB", fg="white", relief="flat")
    btn_bar_chart.pack(side="left", padx=10)

    btn_exit = tk.Button(nav_bar, text="Quitter", command=root.quit, font=("Arial", 12), bg="#FFC0CB", fg="white", relief="flat")
    btn_exit.pack(side="right", padx=10)

    # Section d'affichage de l'image
    panel = tk.Label(root)
    panel.pack(pady=20)

    # Section d'affichage des résultats
    info_label = tk.Label(root, textvariable=info_text, font=("Arial", 12), bg="#FFC0CB")
    info_label.pack(pady=10)

    btn_select_image = tk.Button(root, text="Sélectionner une Image", command=select_image, font=("Arial", 12), bg="#FF6347", fg="white")
    btn_select_image.pack(pady=10)

    btn_predict = tk.Button(root, text="Prédire", command=predict_image, font=("Arial", 12), bg="#32CD32", fg="white")
    btn_predict.pack(pady=10)

    btn_save = tk.Button(root, text="Sauvegarder les Résultats", command=save_results, font=("Arial", 12), bg="#1E90FF", fg="white")
    btn_save.pack(pady=10)

    btn_reset = tk.Button(root, text="Réinitialiser", command=reset, font=("Arial", 12), bg="#FFD700", fg="white")
    btn_reset.pack(pady=10)

    btn_search = tk.Button(root, text="Rechercher une Image", command=search_image, font=("Arial", 12), bg="#FF6347", fg="white")
    btn_search.pack(pady=10)

    root.mainloop()

# Fenêtre de connexion
login_window = tk.Tk()
login_window.title("Connexion")
login_window.geometry("400x300")

# Ajouter une image de fond
background_image = Image.open("logo2.jpg")
background_image = background_image.resize((400, 300))  # Adapter à la taille de la fenêtre
background_image_tk = ImageTk.PhotoImage(background_image)

background_label = tk.Label(login_window, image=background_image_tk)
background_label.place(x=0, y=0, relwidth=1, relheight=1)  # Remplir toute la fenêtre

username_label = tk.Label(login_window, text="Nom d'utilisateur", font=("Arial", 12))
username_label.pack(pady=10)

username_entry = tk.Entry(login_window, font=("Arial", 12))
username_entry.pack(pady=10)

password_label = tk.Label(login_window, text="Mot de passe", font=("Arial", 12))
password_label.pack(pady=10)

password_entry = tk.Entry(login_window, font=("Arial", 12), show="*")
password_entry.pack(pady=10)

login_button = tk.Button(login_window, text="Se connecter", command=login, font=("Arial", 12))
login_button.pack(pady=20)

login_window.mainloop()
